In [ ]:
import torch
import tensorflow as tf
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Dense, Flatten, Dropout
from keras.utils import to_categorical
import sys
import os



In [ ]:
# !wget https://dyslexia.computing.clemson.edu/BET_BSE/BSE_BET_DATA.zip --no-check-certificate

In [ ]:
# !unzip BSE_BET_DATA.zip

In [ ]:
label_file=pd.read_csv("BSE_BET_DATA/labels.csv")

In [ ]:
model = Sequential()

model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu', input_shape=(128, 128, 75, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(Dense(16,activation='relu'))
#model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='sigmoid'))
model.summary()


In [ ]:
model=tf.keras.models.load_model('MRI_Classification_Model_v5.h5')
model.summary()

In [ ]:
# import gzip
# import shutil

# def unzip_gz_files(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".gz"):
#             file_path = os.path.join(folder_path, filename)
#             unzipped_file_path = os.path.join(folder_path, filename[:-3])  # Remove .gz extension
            
#             with gzip.open(file_path, 'rb') as gz_file:
#                 with open(unzipped_file_path, 'wb') as unzipped_file:
#                     shutil.copyfileobj(gz_file, unzipped_file)
#             os.remove(file_path)

# folder_path = "BSE_BET_DATA/files"
# unzip_gz_files(folder_path)

In [ ]:
test_load = nib.load('sample_nii_for_test/la_003.nii').get_fdata()
test_load.shape

In [ ]:



def load_data(data_folder, labels_file):
    images = []
    labels = []
    max_depth = 150
    count=0
   
    for idx in range(len(label_file)):
        count+=1
        print(count)
        line=label_file["Filename"][idx] + " " + str(int(label_file["Recognizable-Facial-Feature"][idx]=="Yes")) + " "+str(int(label_file["Brain-Feature-Loss"][idx]=="Yes"))
        filename, label, labelTwo = line.strip().split()
        filepath = os.path.join(data_folder, filename)
        img = nib.load(filepath).get_fdata()


        
        img = np.pad(img, ((0, 0), (0, 0), (0, max_depth - img.shape[2])),
                     mode='constant')
        img = np.expand_dims(img, axis=3)
        images.append(img)
        

        if(int(label)==0 and int(labelTwo)==0):
            labels.append(0)
        elif(int(label)==1 and int(labelTwo)==0):
            labels.append(1)
        elif(int(label)==0 and int(labelTwo)==1):
            labels.append(2)
        else:
            labels.append(3)

    images = np.array(images).astype(np.float32)
    labels = to_categorical(labels, num_classes=4)
    
    return images, labels

data_folder = "BSE_BET_DATA/files"
labels_file = "BSE_BET_DATA/labels.csv"

images, labels = load_data(data_folder, labels_file)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(images, labels,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
tf.debugging.set_log_device_placement(True)

if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


In [ ]:

history=None
tf.debugging.set_log_device_placement(True)
with tf.device("/GPU:0"):
    model = Sequential()
    model.add(Conv3D(16, kernel_size=(2, 2, 2), activation='relu', input_shape=(256, 256, 150, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(16, kernel_size=(2, 2, 2), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(16, kernel_size=(2, 2, 2), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    
    print(model.count_params())
    
    print("compiling")
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # model.to(device)
    print("training")
    
    history=model.fit(X_train, y_train, batch_size=8, epochs=20, verbose=1, validation_data=(X_test, y_test))

    # Save the model


In [ ]:
model.save("MRI_Classification_Model_v2.h5")

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

y_pred_probs = model.predict(X_test)

y_pred_labels = np.argmax(y_pred_probs, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

print("True labels:", y_true_labels)
print("Predicted labels:", y_pred_labels)

In [ ]:
import pickle

In [ ]:
with open("image_store_2.pkl", "wb") as file:
    pickle.dump(images, file)

In [ ]:
with open("label_store.pkl", "wb") as file:
    pickle.dump(labels, file)

In [ ]:
for val in range(len(labels)):
    finalLabel=[]
    for i in range(3):
        if labels[val][0][i]==1 or labels[val][1][i]==1:
            finalLabel.append(1.0)
        else:
            finalLabel.append(0.0)
    labels[val]=finalLabel

In [ ]:
images=pickle.load(open("image_store_2.pkl", "rb"))

In [ ]:
labels=pickle.load(open("label_store.pkl", "rb"))